In [32]:
!pip install pillow numpy torch opencv-python matplotlib seaborn torchvision

In [61]:
import cv2
import os
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

# Define the folders containing the images
folders = {
    0: "../data/Dataset/Non_Demented/",
    1: "../data/Dataset/Very_Mild_Demented/",
    2: "../data/Dataset/Mild_Demented/",
    3: "../data/Dataset/Moderate_Demented/",
}

# Initialize lists to store the images and labels
X = []
y = []

# Iterate over the folders and load each image
for label, path in folders.items():
    for filename in os.listdir(path):
        # Load the image
        img_path = os.path.join(path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)       
        img_tensor = torch.from_numpy(img).float() 
        X.append(img_tensor)
        y.append(label)

X = torch.stack(X)
# Normalize the images
X = X / 255.0
y = torch.tensor(y)

print(X.shape, y.shape)

# Calculate the count of each class
class_counts = y.bincount()
num_samples = y.size(0)
class_weights = 1. / class_counts

# Create a list of weights for each sample
sample_weights = class_weights[y]

# Set up the sampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

# Create a dataset and a dataloader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)

torch.Size([6400, 128, 128]) torch.Size([6400])


In [65]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 4)  # Assuming 4 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # You can adjust this based on your specific needs

print('starting training')

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(dataloader):
        # Add an extra dimension for the single-channel grayscale image
        images = images.unsqueeze(1)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}')

starting training


 15%|█▌        | 30/200 [00:06<00:37,  4.55it/s]


KeyboardInterrupt: 